- bitflyerのAPI設定
- ティッカーとか板情報の取得

In [1]:
import configparser
import os
import time

import ccxt
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
# API

conf = configparser.ConfigParser()
conf.read('../../../data/bot/settings.ini')

api_key = conf['bitflyer']['api_key']
api_secret = conf['bitflyer']['api_secret']

# CCXT
bitflyer = ccxt.bitflyer()

bitflyer.apiKey = api_key
bitflyer.secret = api_secret

In [8]:
# 5分で300回以上リクエストを送るとエラー
# https://lightning.bitflyer.com/docs?lang=ja#http-private-api

In [5]:
# 口座残高の確認

# bitflyer.private_get_getbalance()

## ticker

In [3]:
params = {
    'product_code': 'BTC_JPY'
}

In [4]:
ticker = bitflyer.public_get_getticker(params=params)
ticker

{'product_code': 'BTC_JPY',
 'state': 'RUNNING',
 'timestamp': '2021-03-17T14:48:57.907',
 'tick_id': 38827729,
 'best_bid': 5991413.0,
 'best_ask': 5992941.0,
 'best_bid_size': 0.03,
 'best_ask_size': 0.01,
 'total_bid_depth': 781.92131423,
 'total_ask_depth': 714.00123975,
 'market_bid_size': 0.0,
 'market_ask_size': 0.0,
 'ltp': 5991413.0,
 'volume': 20168.34288394,
 'volume_by_product': 6102.17743352}

In [5]:
bid = ticker['best_bid']
ask = ticker['best_ask']
ltp = ticker['ltp']

# 実際のspreadと一致することを確認
(ask - bid) / ltp * 100

0.025503165947665433

## 板

In [23]:
# bitflyer.public_get_getboard(params)

## 約定データ(bitflyer版)

In [6]:
# 取得対象
symbol = 'BTC_JPY'
market = 'Spot' # 現物
before = 0      # 初期値

In [ ]:
# 約定データの保存先
path = '../../../data/crypto/bitflyer/executions/'

# すでに持っているデータ
try:
    executions = os.listdir(path+symbol)
except:
    # ディレクトリが無いなら作る
    os.mkdir(path+symbol)
    executions = os.listdir(path+symbol)
    
if '.ipynb_checkpoints' in executions:
    executions.remove('.ipynb_checkpoints')

# TODO:過去に取得してるなら差分だけ取得
if executions:
    print('already have some executions data..')
# まだ約定データを1個も持ってない場合
else:
    print('not have any executions data..')
    df_exec_list = []
    before = 0 # 初期値
    while True:
        params = {
            'product_code': symbol,
            'market': market,
            'count': 500,
            'before': before,
        }
        execution = bitflyer.public_get_getexecutions(params=params)
        # 次のループの取得開始ID
        before = execution[-1]['id']
        df_exec = pd.DataFrame(execution)\
                    .sort_values('id')\
                    .set_index('id')
        df_exec_list.append(df_exec)
        # 約定データ数が溜まったら保存
        # 60万件溜まったら保存
        if len(df_exec_list) == 600000 / 500:
            df_executions = pd.concat(df_exec_list)
            # 昇順にソート
            df_executions = df_executions.sort_values('id', ascending=True)
            start_id = str(df_executions.index[0])
            end_id = str(df_executions.index[-1])
            filename = filename_prefix + '_' + start_id + '_' + end_id + '.csv'
            df_executions['exec_date'] = list(map(lambda x: utcstr2jpystr(x), df_executions['exec_date']))
            df_executions.to_csv(filepath + filename)
            # 初期化
            df_exec_list = []
            print(filename)
        # ループ終了処理
        if True:
            pass
        
        time.sleep(1.0)

not have any executions data..


In [10]:
df_exec.head()

,side,price,size,exec_date,buy_child_order_acceptance_id,sell_child_order_acceptance_id
id,,,,,,
2200758628,SELL,6014126.0,0.01275,2021-03-17T10:11:55.597,JRF20210317-101151-203581,JRF20210317-101155-223836
2200758629,SELL,6013227.0,0.01000,2021-03-17T10:11:55.957,JRF20210317-101141-464303,JRF20210317-101155-100707
2200758630,SELL,6013000.0,0.03000,2021-03-17T10:11:56.303,JRF20210317-101143-438060,JRF20210317-101156-223838
2200758631,SELL,6010900.0,0.09070,2021-03-17T10:11:56.837,JRF20210317-101155-100706,JRF20210317-101156-210108
2200758633,BUY,6014125.0,0.04000,2021-03-17T10:11:57.1,JRF20210317-101156-119189,JRF20210317-101156-219550


In [61]:
before = 0

params = {
        'product_code': symbol,
        'market': market,
        'count': 500,
        'before': before,
    }

## 板データ収集
過去の板データを分析したい